# Functools Module


The functools module, part of Python’s standard Library, provides useful features that make it easier to work with high order functions (a function that returns a function or takes another function as an argument ). With these features, you can reuse or extend the utility of your functions or callable object without rewriting them. This makes the writing of reusable and maintainable code to be quite simple.

As per the current stable release i.e., Python 3.8 series, the functools module contains 11 funtions and some of these may not be available or work differently on earlier or later releases. They include:
* reduce()
* lru_cache()
* partial()
* partialmethod()
* singledispatch()
* singledispatchmethod()
* cached_property()
* total_ordering()
* update_wrapper()
* wraps()
* cmp_to_key()

## Lambda


A lambda function is a small anonymous function.

A lambda function can take any number of arguments, but can only have one expression.

### Syntax

> lambda arguments : expression

In [2]:
x = lambda a, b: a+b
print(x(4,5))

9


## Why Use Lambda Functions?

The power of lambda is better shown when you use them as an anonymous function inside another function.

Say you have a function definition that takes one argument, and that argument will be multiplied with an unknown number:

In [5]:
## myFunc return a function object
def myFunc(n):
    return lambda a : a*n

myDoubler = myFunc(2)
print(myDoubler(11))

22


### 1. reduce()

The reduce(function, sequence) function receives two arguments, a function and an iterable. It applies the argument function cumulatively to all elements of the iterable from the left to the right and then returns a single value.
To put it simply, it first applies the argument function to the first two elements of the iterable and the value returned by this first call becomes the function’s first argument and the third element of the iterable becomes the second argument. This process is repeated until the iterable is exhausted.

In [6]:
from functools import reduce
ls=[1,2,3,4]
list_sum = reduce(lambda a, b: a+b, ls)
print(list_sum)

10


In [7]:
list_multiple = reduce(lambda a, b: a*b, ls)
print(list_multiple)

24


### 2. lru_cache()

lru_cache() is a decorator, which wraps a function with a memoizing callable used for saving up to maxsize the results of a function call and returns the stored value if the function is called with the same arguments again. It can save time when an expensive or I/O bound function is periodically called with the same arguments.
Essentially it uses two data structures, a dictionary to map a function’s parameters to its result, and a linked list to keep track of the function’s call history.
In full LRU Cache stands for Least-Recently-Used Cache and refers to a cache which drops the least recently used element if the maximum size of entries is reached. The LRU feature is disabled if maxsize is set to None and caches arguments of different data types separately if typed is True e.g., f(3) and f(3.0) will be cached distinctly.

An example of the utility of lru_cache() can be shown in optimizing code that generates the factorial of a number

In [10]:
from functools import lru_cache

def uncached_factorial(n):
    if n < 1:
        return 1
    return n * uncached_factorial(n-1)

## evaluate the performence 
%timeit uncached_factorial(100)

14.9 µs ± 531 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [18]:
@lru_cache(maxsize=None)
def cached_factorial(n):
    if n == 0:
        return 1
    if n == 1:
        return 1
    return n * uncached_factorial(n)
%timeit cached_factorial(100)

79.5 ns ± 6.86 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Without @lru_cache the factorial function takes around 1.46 µs to run while, on the other hand, with @lru_cache, the function only takes 158 ns. This equates to almost a 100,000 times improvement in performance- Amazing! Right?

In general, the LRU cache should only be used when you want to reuse previously computed values. Accordingly, it doesn’t make sense to cache functions that need to create distinct mutable objects on each call. Also, since a dictionary is used to cache results, the positional and keyword arguments to the function must be hashable.

## 3. partial()

Partial functions are derived functions that have some pre-assigned input parameters. For example, if a function takes in two parameters say “a” and “b”, a partial function can be created from it that has “a” as a prefilled argument and it can then be called with “b” as the only parameter. Functool’s partial() is used to create partial functions/objects and this is a useful feature as it allows for the:
* Replication of existing functions with some arguments already passed in.
* Creation of newer version of the existing function in a well-documented manner.
Let’s consider a simple example to illustrate this

In [24]:
from functools import partial
import math

permutaion_of_nine = partial(math.perm, 9) ## math.perm(n,k) choose K from n, math.perm(7, 5) = 7*6*5*4*3 = 2520
permutaion_of_nine.__doc__ = '''Returns the number of ways to choose k items f'''
permutaion_of_nine.__name__ = "premutation_of_nine"

print(permutaion_of_nine(2))

72


We first create a partial object based on math.perm() function. In this case, we set 9 as the first argument. Consequently, the newly created permutation_of_nine function behaves as if we call math.perm() with 9 set as the default parameter. In our example, permutation_of_nine(2) does the same thing as math.perm(9,2).

It is important to note that the __name__ and __doc__ attributes are to be specified by the programmer as they are not created automatically

The partial function also comes with important attributes that prove to be useful in tracking partial functions/objects. These include:

partial.args — Which returns the positional arguments preassigned to the partial function.
partial.keywords — Which returns the keyword arguments preassigned to partial function.
partial.func — Which returns the name of parent function along with its address.
Let us look at another that illustrates these features

In [26]:
## parent function
def adder(x, y):
    return x + y

## partial function

add2 = partial(adder, 2)
print(add2.args)
add9 = partial(adder, 9)

(2,)


Partials are incredibly useful. For example, in a pipe-lined sequence of function calls in which the returned value from one function is the argument passed to the next.

## 4. partialmethod()
The partialmethod() returns a new partialmethod descriptor which behaves like partial except that it is designed to be used as a method definition rather than being directly callable. You can think of it as the partial() for methods.

Perhaps an example is best suited to illustrate this.

In [42]:
from functools import partialmethod

class Animal:
    def __init__(self):
        self.species ="Bear"
    def _set_species(self, species):
        self.species = species
    set_dog = partialmethod(_set_species,species="Dog")
    set_rabbit = partialmethod(_set_species, species="Rabbit")
    
animal = Animal()
print(f'Default animal is:{animal.species}')
animal.set_dog()
print(animal.species)

Default animal is:Bear
Dog


We first create an Animal class that has an attribute species and an instance method _set_species() that sets the animal’s species. Next, we create two partialmethod descriptors set_dog() and set_rabbit(), which call _set_species() with “Dog” or “Rabbit”, respectively. This allows us to create a new instance of the class Animal, call set_dog() to change the animal’s species to Dog and finally print the new attribute.

## 5.singledispatch()
Before we discuss this function, it is important that we first gloss over two important concepts:

1. The first one is a generic function which is a function composed of multiple functions implementing the same operation for different types. The implementation to be used during a call is determined by the dispatch algorithm.
2. The second is the Single dispatch, which is a form of a generic function dispatch where the implementation is chosen based on the type of a single argument.

With this in mind, the functool’s singledispatch is a decorator that transforms a simple function into a generic function whose behaviour is dependent on the type of its first argument. In plain language, it is used for function overloading

Let see an example of it in action.

In [4]:
from functools import singledispatch

@singledispatch
def devide_i(a:int, b:int)->float:
    return a/b
@devide_i.register(str)
def _(a:str, b:str) ->str:
    return f'{a}/{b}'

print(devide_i(4,2))
print(devide_i('4', '2'))


2.0
4/2


We first define a function divide() that takes two arguments a and b and returns the value of a/b. However, dividing strings will result in a TypeError and to deal with this we define the _ functions which specifies the behaviour of divide() if it is supplied with strings. Note that the overloaded implementations are registered using the register() attribute of the generic function

## 6. singledispatchmethod()
It is a decorator that does the exact thing as @singledispatch but it is specified for methods rather than functions.

Consider the following example.

In [23]:
from functools import singledispatchmethod
import math
from typing import List, Set

class Product:
    @singledispatchmethod
    def prod_n(self, args):
        raise NotImplementedError(f'cannot multiply {args}')
    
    @prod_n.register
    def _(self, args:List[int])->int:
        return math.prod(args)
    
    @prod_n.register
    def _(self, args:Set[int])->int:
        return math.prod(args)
    
    product = Product()
    #print(product.prod_n([1,2,3,4,5]))
    #print(product.prod_n({6,7,8,9,10}))

TypeError: Invalid annotation for 'args'. typing.List[int] is not a class.

The prod method of the Product class is overloaded to return the product of the elements of a list or a set but if supplied with a different type it, by default, raises a NotImplementedError.

## 7. cached_property()
As the name suggests the cached_property() is a decorator that transforms a class method into a property whose value is calculated only once and then cached as a normal attribute for the life of the instance. It is similar to @property except the for its caching functionality. It is useful for computationally expensive properties of instances that are otherwise effectively permanent.

In [5]:
from functools import cached_property
import statistics

class DataSet:
    def __init__(self, sequence_of_numbers):
        self._data = sequence_of_numbers
        
    @cached_property
    def stdev(self):
        return statistics.stdev(self._data)
    
    @cached_property
    def variance(self):
        return statistics.variance(self._data)
observations = DataSet([50,60,70,80,90,100])
print(observations.stdev)
print(observations.variance)

18.708286933869708
350


In the example above, we have a DataSet class that holds a list of observations and implements methods to calculate the variance and standard deviation. The problem is that every time the methods are called the variance and standard deviations would have to be re-calculated and this might prove to be expensive especially for large datasets. @cached_property mitigates this problem by calculating and storing the value only once and returns it if the method is called again by the same instance.

## 8. total_ordering()
Given a class defining one or more rich comparison ordering methods i.e., __lt__(), __le__(), __gt__(), __ge__() or __eq__() (corresponding to <, <=, >, >=, and ==). You can define a few of the comparison methods, and @total_ordering will automatically supply the rest as per the given definitions. It is important that the class should supply an __eq__() method.

For example, if you want to create a class that compares different numbers. You would probably need to implement all of the rich comparison methods.However, this might be quite tedious and redundant, to solve this you can only implement the __eq__ and the __gt__ method and use @total_ordering to automatically fill up the rest.

In [3]:
from functools import total_ordering

@total_ordering
class CompareNum:
    def __init__(self, value):
        self.value = value
        
    def __eq__(self, new_value):
        return self.value == new_value.value
    
    def __gt__(self, new_value):
        return self.value > new_value.value
    
print(CompareNum(4) > CompareNum(9))
print(CompareNum(4) < CompareNum(9))
    

False
True


## 9. update_wrapper()
It updates the metadata of a wrapper function to look like the wrapped function. For example, in the case of partial functions, update_wrapper(partial, parent) will update the documentation(__doc__) and name(__name__) of the partial function to match that of the parent function.

In [20]:
from functools import update_wrapper, partial

## parent function
def power(a, b):
    ''' a to the power b '''
    return a ** b
print(power(2,3))
print(power(3,2))

power2 = partial(power, b=2)
print(power2(3))
print(power2.__doc__)
print(power2.__name__)

8
9
9
partial(func, *args, **keywords) - new function with partial application
    of the given arguments and keywords.



AttributeError: 'functools.partial' object has no attribute '__name__'

In [21]:
update_wrapper(power2, power)
print(power2.__doc__)
print(power2.__name__)

 a to the power b 
power


## 10. wraps()
It is a convenience function for invoking update_wrapper() to the decorated function. It is equivalent to running partial(update_wrapper, wrapped=wrapped, assigned=assigned, updated=updated).

In [2]:
from functools import wraps

def my_decorator(f):
    @wraps(f)
    def wrapper(*args, **kwds):
        print("Calling decorated function")
        return f(*args, **kwds)
    return wrapper

@my_decorator
def example():
    print("calling example function...")
    
example()

Calling decorated function
calling example function...


## 11. cmp_to_key()
It transforms an old-style comparison function to a key function. A comparison function is any callable that accepts two arguments, compares them, and returns a negative number for less-than, zero for equality, or a positive number for greater-than. Whereas a key function is a callable that accepts one argument and returns another value to be used as the sort key, an example is the operator.itemgetter() key function. Key functions are used in tools such as sorted(), min(), max() and itertools.groupby().

cmp_to_key() is majorly used as a transition tool for programs written in Python 2 that support comparison functions.

Let’s take an example of how we can use a comparison function to sort a list of strings according to the first letter to illustrate the use of cmp_to_key()

In [3]:
from functools import cmp_to_key

## first we define a comparasion function
def comparison(a, b):
    if a[0] < b[0]:
        return -1
    elif a[0] == b[0]:
        return 0
    else:
        return 1
    
name = ["Larry", "Alex", "Berry", "David"]
sorted_name = sorted(name, key=cmp_to_key(comparison))
print(sorted_name)

['Alex', 'Berry', 'David', 'Larry']
